# 5 - Lambdas and Closures
##### **Author: Adam Gatt**
"Lambdas" and the related "closures" are hugely useful concepts borrowed from [Functional Programming](https://en.wikipedia.org/wiki/Functional_programming). In the shortest explanation, a "lambda" is an anonymous function and a "closure" is a lambda that can include state from its environment at the time of its creation. If this makes sense to you, you might already be familiar with FP. But for this notebook we will work our way up to lambdas, by stepping through the pre-11 concepts of function references and functors.

## Function pointers and references
Many computational tasks involve the use of a _"High Level Function"_, a function that accepts another function as a parameter. Common examples of this might be:
* A function needs a _"callback"_ to be provided that it will execute when it fetches the data it needs.
* A sort function needs a "comparator" function to tell it which of two elements should come first.
* A filter function needs a "predicate" function to tell it which elements to keep.
* A timer function that accepts another function to be periodically executed.
* A profiler function that sets up a clock and times how long it takes a supplied function to execute.
* The result of creating a function to "factor-out" or "generalise" two very similar pieces of code that differ by some small key piece of functionality. An example of this is shown below:

In [4]:
// Original function - filter a list of ints to just those divisible by 2
std::vector<int> filterMultiplesOfTwo(std::vector<int> input);

// Generalise out the value "2" - now we filter to those divisible by any number we specify
std::vector<int> filterMultiples(std::vector<int> input, int base);

// A reference to a "predicate" function that accepts an element and decides true/false
using Predicate = bool(&)(int);

// Generalise out the concept of checking multiples - now we filter to those ints that pass
// whatever "predicate function" we provide to our high-level function.
std::vector<int> filter(std::vector<int> input, Predicate pred);

C++ readily allows us to create pointers and references to functions like we can for variables. We can assign these references to variables, re-assign them, pass them to/from functions, etc.

In [1]:
int timesTwo(int input) {
    return input * 2;
};

// "doubler" is a reference to the function "timesTwo". The type of this
// reference is a little messy - return_type(&name)(parameters). Note
// that the brackets around doubler are very important
int (&doubler)(int) = timesTwo;

// Using "auto" may be cleaner
auto &doubler2 = timesTwo;

// We can call our function reference, the end result is the same as
// if we called the original function
doubler(5)

10

In [2]:
// We could also use pointers instead of references if there is a good reason to
int (*doubler3)(int);

doubler3 = &timesTwo;

doubler3(-8)

-16

To submit a function reference to another function we first have to write a full function definition and so that we can reference it in the higher-level function call. This can be a lot of extra boilerplate if we are only calling the higher-level function once, and using the inner function nowhere else.

These inner function definitions need to be written somewhere. If we make frequent use of higher-level functions then we will end up with large collections of these once-off function definitions, bloating our design.

In [2]:
#include <algorithm>

// I give this function a name here
bool isHorror(Book book) {
    return book.genre == "Horror";
}

// Just to use it in one actual place - here
auto horrorBookItr = std::find_if(bookList.begin(), bookList.end(), &isHorror)

This seems to be unique for function parameters. We can use other types of parameters, such as integers, by simply providing an actual literal value directly to the function call.

In [ ]:
// 255 is an "integer literal" that we can specify directly to an integer parameter.
int clippedValue = std::min(originalValue, 255);

// "Adam" is "string literal" that we can specify directly to a (const char*)
// parameter (or std::string via implicit conversion).
auto foundPeople = std::find(names.begin(), names.end(), "Adam");

## Lambda - the Function Literal
[_"Lambdas"_](https://en.wikipedia.org/wiki/Anonymous_function) are a concept from Functional Programming (and originally from [Lambda Calculus](https://en.wikipedia.org/wiki/Lambda_calculus)) that are supported in a number of languages and were introduced to C++ in C++11. In short, a lambda is an "anonymous function"; a function body that hasn't been assigned to a name.

The syntax for a lambda looks as follows. Note the different types of brackets.
> `[capture_list](parameters){function_body}`

For example:

In [3]:
// This has input parameters, a body and a return statement just like a function, but
// nowhere has it actually been given a name
[](int x, int y){return x + y;};

Don't worry about the opening `[]` for now; we'll cover it in a moment. But note how this looks just like a function without a name. We can put as many statements as we want within the curly braces. It's just that the entire section where the name, return type and qualifiers would all go is simply replaced by the `[]`.

I like to think of lambdas as "function literals", in much the same way that other data types also have literals. If we look at the statement
> `int age = 20;`

we know that `age` is an int. But `20` is also an int, hence why we can assign it to age or supply it directly as an argument to int parameters. 20 is a particular literal value of the int data type - an integer literal. Although we assign it to a variable, it can exist perfectly well without a name, for example in expressions such as `(20 - 5) / 3.0f`.

In [1]:
#include <string>

int age = 20;

char key = 'x';

double distance = 1.8;

float progress = 2.5f;

std::string greeting = "Hello";

// The right-hand side is a lambda expression, containing a particular literal function expression.
auto square = [](int input){ return input * input; };

square(20)

400

Now that we can represent a function literal then we can finally submit a function to a parameter without needing to write a full definition somewhere else first. Here is our horror book finder from earlier.

#### Lambdas can be used without ever giving them a name

In [2]:
// This function expect us to provide an int
void intPrinter(int value) {
    printf("The value is: %d", value);
}

// We know we can supply an int-literal. We don't need to assign 10
// to a variable first in order to give it to the function.
intPrinter(10);

The value is: 10

In [1]:
#include <algorithm>

auto horrorBookItr = std::find_if(bookList.begin(), bookList.end(), [](Book book){return book.genre == "Horror";});

input_line_8:2:73: error: unknown type name 'Book'
 auto horrorBookItr = std::find_if(bookList.begin(), bookList.end(), [](Book book){return book.genre == "Horror";});
                                                                        ^
input_line_8:2:36: error: use of undeclared identifier 'bookList'
 auto horrorBookItr = std::find_if(bookList.begin(), bookList.end(), [](Book book){return book.genre == "Horror";});
                                   ^
input_line_8:2:54: error: use of undeclared identifier 'bookList'
 auto horrorBookItr = std::find_if(bookList.begin(), bookList.end(), [](Book book){return book.genre == "Horror";});
                                                     ^


Interpreter Error: 

We can use as many lambdas as we need without needing to write a bunch of function definitions. We can substitute in different lambdas to provide different behaviour to the high-level function.

In [1]:
#include <vector>
#include <string>

struct Film {
    std::string title;
    int yearReleased;
    std::string rating;
};

std::vector<Film> collection {{"a", 2, "b"}, {"c", 4, "d"}};

// Do I want to sort by title?
std::sort(collection.begin(), collection.end(), [](const Film& a, const Film& b){return a.title < b.title;});

// Or year or rating. Whatever the current operation requires.
std::sort(collection.begin(), collection.end(), [](const Film& a, const Film& b){return a.yearReleased < b.yearReleased;});
std::sort(collection.begin(), collection.end(), [](const Film& a, const Film& b){return a.rating < b.rating;});

This "Immediately Invoked Function Expression" doesn't need an assignment or a name in order to be executed.

In [2]:
#include <iostream>
#include <string>

std::cout << [](std::string input){return input.size();} ("Lambda") << std::endl;

6


Lambdas find great use in GUI applications like Qt where GUI elements can trigger events (button clicks, sliders moving). The developer can use a lambda to submit a little script that should run when the event is triggered, a considerable efficiency over registering a subscriber/listener or writing an anonymous inner class. These are traditional boilerplate-heavy solutions to what is essentially just describing a piece of processing functionality.

In [ ]:
QPushButton messageButton{"Test Debug"};

connect(messageButton, &QPushButton::clicked, [](){qDebug() << "Test Message";});

## Closures and the capture list
The square brackets at the start of the lambda syntax is the _"capture list"_. You can list existing variables here and they will be available for use in the lambda body.

In [1]:
#include <vector>
#include <functional>

// Dummy higher-order function - fetches a name from a database and provides it to a callback
void getNameFromDB(std::function<void(std::string)> callback) {
    std::string foundRecord = "Michael";
    
    callback(foundRecord);
}


std::list<std::string> names;

// "names" is from an outer scope to the lambda body. The lambda only knows about it from its capture list.
// The "&" indicates that we are using a reference for "names" instead of making a copy
getNameFromDB( [&names](std::string newName){names.push_back(newName);} );

names

{ "Michael" }

Technically this makes the lambda function into a [_closure_](https://en.wikipedia.org/wiki/Closure_(computer_programming)). This is a lambda that "encloses" variables from its surrounding scope so that it can continue to have access to them no matter where it is actually invoked (even in a different scope).

In [1]:
#include <iostream>
#include <functional>

std::function<double(double)> getScaler(double factor) {
    return [factor](double input){return input * factor;};
}

auto scaleByFour = getScaler(4.0);

auto scaleByTen = getScaler(10.0);

// The "factor" variable only existed within the scope of the getScaler function, but by
// capturing it in the lambda and then returning that lambda we have smuggled out these
// values to be used in whatever scope the lambda is eventually called in.
std::cout << scaleByFour(6.0) << " - " << scaleByTen(6.0);

24 - 60

Conceptually, it can help to think of the difference this way:
 * Treat a **lambda** like a "function" in mathematics - it should only really interact with input parameters and have no "side effects".
 * Treat a **closure** like a function that can be passed around and still interact with variables from the scope it was created.

### Capture List Syntax

Variables can be captured either by value (copied into the closure) or by reference. When copying by reference, you must be careful that the referenced variable will continue to exist at the time that the closure is executed, in order to avoid a dangling reference error.

| Syntax | Description |
| --- | --- |
| \[ = \<var\> \] | Capture variable by value |
| \[ & \<var\> \] | Capture variable by reference |

It is possible to capture multiple variables by listing them all out. There is also a convenience syntax of simply writing `[=]` to capture all variables that appear in the closure body by value, or `[&]` to capture them by reference.

| Syntax | Description |
| --- | --- |
| \[ \<var1\>, \<var2\>, ... \] | Capture multiple variables |
| \[ = \] | Automatic capture of variable values |
| \[ & \] | Automatic capture of variable references |


If the closure appears within the context of a class object, you will need to capture `this` to be able to use the object's data members. This can be included by the automatic capture syntax but it will always capture the object by a pointer (the pointer is copied by value) regardless of the automatic capture approach chosen. A by-value copy of the entire object can be captured in C++17 onwards by capturing `*this`;

| Syntax | Description |
| --- | --- |
| \[ this \] | Capture the current object pointer |
| \[ *this \] | Capture the current object by value |

XXX REDEFINITION OF CAPTURE VARIABLES TO DIFFERENT NAMES XXX

### Closures are a sleek alternative to Functors
The need for an inner function that accesses its originating scope is traditionally fulfilled in C++ by the use of the _Functor_ idiom. A Functor is a class that implements `operator()` so that instances of it can be "called" as though they were functions. But being fully-fledged classes they can accept arbitrary references and values at the site of their instantiation (via the constructor) and store them as data members for use when the functor is eventually called. An example of this is below:

In [1]:
#include <fstream>
#include <string>

// Classic functor that accepts data in its constructor, remembers it and uses it
// in the operator() method
class ListLogger {
    private:
        std::list<std::string>& log;
    public:
        ListLogger(std::list<std::string>& log) : log(log) { }
    
        void operator()(std::string message) {
            log.push_back(message);
        }
};

In [2]:
// With our functor instance we can call it with a syntax that looks identical to a function call

template <typename T>
void executeTestProcedure(T logger) {
    logger("Starting tests");
    // Do some stuff
    logger("Tests finished");
}

In [3]:
std::list<std::string> myLogList;

ListLogger myLogger{myLogList};

executeTestProcedure(myLogger);

myLogList

{ "Starting tests", "Tests finished" }

Based on how it is used, we can see that a Functor is the OO approach to a closure, a defined piece of processing functionality that can interact with the scope in which it was created. Its full class definition results in a large amount of boilerplate that is not necessary for the closure equivalent.

In [5]:
auto closureLogger = [&myLogList](std::string message) {
    myLogList.push_back(message);
};

executeTestProcedure(closureLogger);

myLogList

input_line_15:2:25: error: 'myLogList' cannot be captured because it does not have automatic storage duration
 auto closureLogger = [&myLogList](std::string message) {
                        ^
input_line_10:2:25: note: 'myLogList' declared here
 std::list<std::string> myLogList;
                        ^


Interpreter Error: 

XXX ? the strategy pattern ? XXX

## What is the type of lambdas and closures?

 * can submit lambda to a function reference/pointer parameter but not closure
 * can use std::functional (performance cost?)
 * can use template function or "auto" in generalised lambda